Ejercicio 1

In [1]:
import cv2

In [ ]:
if __name__ == "main":

img1 = cv2.imread('img_01.jpg', cv2.IMREAD_GRAYSCALE)
x1 = img_to_vect(img1)
graficar_vector(x1, img1)

img2 = cv2.imread('img_02.jpg', cv2.IMREAD_GRAYSCALE)
x2 = img_to_vect(img2)
graficar_vector(x2, img2)

IndentationError: expected an indented block after 'if' statement on line 1 (2765464670.py, line 3)